<a href="https://colab.research.google.com/github/helonayala/sysid/blob/main/frols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forward-regression orthogonal least squares

The method consists of using OLS for ranking the model candidate terms by ERR.

The example below implements the Example 3.6 given in the book by Billings, 2013.

## Imports and helper functions

In [3]:
import numpy as np
from itertools import combinations_with_replacement
import pandas as pd # Optional: for prettier display in the example

# Helper function to create the basic ARX matrix
def regMatARX(y_signal_in,
              u_signal_in,
              ny: int,
              nu: int):
    """
    Creates the initial ARX regression matrix and the target vector y(k).
    AR (y-lag) terms are NOT negated.

    Args:
        y_signal_in (array-like): Output data vector.
        u_signal_in (array-like): Input data vector.
        ny (int): Number of past y lags (autoregressive order).
        nu (int): Number of past u lags (exogenous input order).

    Returns:
        P0_data (np.ndarray): The ARX regressor matrix. Shape (NP, ny + nu).
        P0_colnames (list): Column names for P0_data.
        y_target (np.ndarray): The target vector y(k). Shape (NP,).
    """
    if not isinstance(y_signal_in, np.ndarray):
        y_signal = np.array(y_signal_in, dtype=float)
    else:
        y_signal = y_signal_in.astype(float)

    if not isinstance(u_signal_in, np.ndarray):
        u_signal = np.array(u_signal_in, dtype=float)
    else:
        u_signal = u_signal_in.astype(float)

    if len(y_signal) != len(u_signal):
        raise ValueError("Input signals y_signal and u_signal must have the same length.")
    if ny < 0 or nu < 0:
        raise ValueError("Lags ny and nu must be non-negative.")

    N_total_samples = len(y_signal)

    max_lag = 0
    if ny > 0:
        max_lag = max(max_lag, ny)
    if nu > 0:
        max_lag = max(max_lag, nu)

    # Target vector y(k)
    # These are the y values that each row of regressors corresponds to.
    y_target = y_signal[max_lag:]
    num_effective_rows = len(y_target)

    P0_colnames = []
    if ny > 0:
        P0_colnames.extend([f'y(k-{i})' for i in range(1, ny + 1)])
    if nu > 0:
        P0_colnames.extend([f'u(k-{i})' for i in range(1, nu + 1)])
    num_P0_cols = len(P0_colnames)

    if num_effective_rows == 0: # Not enough data to form any rows
        return np.empty((0, num_P0_cols), dtype=float), P0_colnames, np.empty((0,), dtype=float)

    P0_rows_list = []
    for k_target_idx in range(max_lag, N_total_samples):
        current_regressor_row = []
        # Add y lags: y(k-j) means y_signal[k_target_idx - j_lag] (NO NEGATION)
        for j_lag_idx in range(1, ny + 1):
            current_regressor_row.append(y_signal[k_target_idx - j_lag_idx])

        for j_lag_idx in range(1, nu + 1):
            current_regressor_row.append(u_signal[k_target_idx - j_lag_idx])
        P0_rows_list.append(current_regressor_row)

    P0_data = np.array(P0_rows_list, dtype=float)

    return P0_data, P0_colnames, y_target


def regMatNARX(u_signal_in,
               y_signal_in,
               nu: int,
               ny: int,
               poly_order_l: int):
    """
    Generates the regression matrix for a NARX model with power-form polynomial terms,
    and the corresponding target vector y(k).
    AR (y-lag) terms in the base regressors are NOT negated.

    Args:
        u_signal_in (array-like): Input data vector.
        y_signal_in (array-like): Output data vector.
        nu (int): Order of the X part (number of u lags).
        ny (int): Order of the AR part (number of y lags).
        poly_order_l (int): Maximum order of the polynomial terms. Must be >= 1.

    Returns:
        P_final_data (np.ndarray): The final NARX regression matrix.
        P_final_colnames (list): Column names for P_final_data.
        y_target (np.ndarray): The target vector y(k).
    """
    if not isinstance(y_signal_in, np.ndarray):
        y_signal = np.array(y_signal_in, dtype=float)
    else:
        y_signal = y_signal_in.astype(float)

    if not isinstance(u_signal_in, np.ndarray):
        u_signal = np.array(u_signal_in, dtype=float)
    else:
        u_signal = u_signal_in.astype(float)

    if len(y_signal) != len(u_signal):
        raise ValueError("Input signals y_signal and u_signal must have the same length.")
    if ny < 0 or nu < 0:
        raise ValueError("Lags ny and nu must be non-negative.")
    if poly_order_l < 1:
        raise ValueError("Polynomial order 'poly_order_l' must be at least 1.")

    # Create the initial ARX regressors and target vector
    # Note: regMatARX expects (y, u, ny, nu) order
    P0_data, P0_colnames, y_target = regMatARX(y_signal, u_signal, ny, nu)

    NP = len(y_target) # Number of effective rows

    if NP == 0:
        P_columns_list = [np.empty((0, 1), dtype=float)] # Constant term
    else:
        P_columns_list = [np.ones((NP, 1), dtype=float)] # Constant term

    P_final_colnames = ['constant']

    P_columns_list.append(P0_data)
    P_final_colnames.extend(P0_colnames)

    num_P0_base_regressors = P0_data.shape[1]

    if poly_order_l >= 2 and num_P0_base_regressors > 0:
        for current_poly_order in range(2, poly_order_l + 1):
            for col_indices_tuple in combinations_with_replacement(range(num_P0_base_regressors), current_poly_order):
                selected_P0_cols_for_product = P0_data[:, list(col_indices_tuple)]
                new_poly_term_column = np.prod(selected_P0_cols_for_product, axis=1, keepdims=True)
                P_columns_list.append(new_poly_term_column)

                term_name = "".join([P0_colnames[i] for i in col_indices_tuple])
                P_final_colnames.append(term_name)

    if not P_columns_list:
        P_final_data = np.empty((0, len(P_final_colnames)), dtype=float)
    else:
        P_final_data = np.concatenate(P_columns_list, axis=1)

    return P_final_data, P_final_colnames, y_target

Next we run some examples for synthetic data of ARX and NARX (power-form polynomials) regression matrices.

In [9]:
# Sample data
u_data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
y_data = np.array([1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7]) # y(0) to y(7)

# Model parameters for NARX
nu_order_narx = 5  # u(k-1)
ny_order_narx = 5  # y(k-1), y(k-2)
poly_degree_narx = 2 # Up to quadratic terms

# Generate the NARX regression matrix and target vector
# Calling regMatNARX(u, y, nu, ny, l)
narx_matrix, narx_colnames, narx_y_target = regMatNARX(
    u_data, y_data, nu_order_narx, ny_order_narx, poly_degree_narx
)

print("--- NARX Model ---")
print(f"Target y(k) vector (shape {narx_y_target.shape}):")
print(narx_y_target) # Should be y_data[max_lag:] = y_data[2:] = [1.2, 1.3, ..., 1.7]

# Optional: Display using pandas DataFrame
# Create a DataFrame for regressors
df_narx_regressors = pd.DataFrame(narx_matrix, columns=narx_colnames)
# Add the target vector as the first column for easy inspection
df_narx_full = pd.concat(
    [pd.DataFrame(narx_y_target, columns=['y(k)_target']), df_narx_regressors],
    axis=1
)
print("\nNARX Regression Matrix (P) and Target y(k):")
print(df_narx_full)


--- NARX Model ---
Target y(k) vector (shape (3,)):
[1.5 1.6 1.7]

NARX Regression Matrix (P) and Target y(k):
   y(k)_target  constant  y(k-1)  y(k-2)  y(k-3)  y(k-4)  y(k-5)  u(k-1)  \
0          1.5       1.0     1.4     1.3     1.2     1.1     1.0     0.5   
1          1.6       1.0     1.5     1.4     1.3     1.2     1.1     0.6   
2          1.7       1.0     1.6     1.5     1.4     1.3     1.2     0.7   

   u(k-2)  u(k-3)  ...  u(k-2)u(k-2)  u(k-2)u(k-3)  u(k-2)u(k-4)  \
0     0.4     0.3  ...          0.16          0.12          0.08   
1     0.5     0.4  ...          0.25          0.20          0.15   
2     0.6     0.5  ...          0.36          0.30          0.24   

   u(k-2)u(k-5)  u(k-3)u(k-3)  u(k-3)u(k-4)  u(k-3)u(k-5)  u(k-4)u(k-4)  \
0          0.04          0.09          0.06          0.03          0.04   
1          0.10          0.16          0.12          0.08          0.09   
2          0.18          0.25          0.20          0.15          0.16   

   u(k-4)u

In [10]:
# --- Example for regMatARX (Linear ARX) ---
print("\n\n--- ARX Model (Linear Terms Only) ---")
ny_order_arx = 2
nu_order_arx = 1
# Calling regMatARX(y, u, ny, nu)
arx_P0_matrix, arx_P0_colnames, arx_y_target = regMatARX(
    y_data, u_data, ny_order_arx, nu_order_arx
)

print(f"Target y(k) vector for ARX (shape {arx_y_target.shape}):")
print(arx_y_target)

df_arx_P0_regressors = pd.DataFrame(arx_P0_matrix, columns=arx_P0_colnames)
df_arx_full = pd.concat(
    [pd.DataFrame(arx_y_target, columns=['y(k)_target']), df_arx_P0_regressors],
    axis=1
)
print("\nARX Regression Matrix (P0) and Target y(k):")
print(df_arx_full)




--- ARX Model (Linear Terms Only) ---
Target y(k) vector for ARX (shape (6,)):
[1.2 1.3 1.4 1.5 1.6 1.7]

ARX Regression Matrix (P0) and Target y(k):
   y(k)_target  y(k-1)  y(k-2)  u(k-1)
0          1.2     1.1     1.0     0.2
1          1.3     1.2     1.1     0.3
2          1.4     1.3     1.2     0.4
3          1.5     1.4     1.3     0.5
4          1.6     1.5     1.4     0.6
5          1.7     1.6     1.5     0.7


## FROLS implementation

Next we write the FROLS function to proceed with term selection of NARX polynomial full models. This has been translated from R to python.

In [12]:
# --- FROLS Function ---
def frols(P_regressors, Y_target_in, rho, P_colnames=None, epsilon=1e-12):
    """
    Forward Orthogonal Least Squares algorithm for model term selection and parameter estimation.

    Args:
        P_regressors (np.ndarray): Candidate regressor matrix (NP, M).
        Y_target_in (np.ndarray): Target vector (NP,) or (NP,1).
        rho (float): Error reduction ratio threshold for stopping.
        P_colnames (list, optional): Names of columns in P_regressors.
        epsilon (float, optional): Small value for numerical stability.

    Returns:
        dict: A dictionary containing selected terms, parameters, and other metrics.
              Keys: 'th', 'Psel_data', 'Psel_colnames', 'g', 'W', 'A',
                    'ERR_values', 'selected_indices'.
    """
    if Y_target_in.ndim == 1:
        Y_target = Y_target_in.reshape(-1, 1)
    else:
        Y_target = Y_target_in

    M = P_regressors.shape[1]  # Number of candidate regressors
    NP = P_regressors.shape[0] # Number of data points

    # Handle edge cases
    empty_result = {
        'th': np.array([]), 'Psel_data': np.empty((NP,0)), 'Psel_colnames': [],
        'g': np.array([]), 'W': np.empty((NP,0)), 'A': np.empty((0,0)),
        'ERR_values': np.array([]), 'selected_indices': []
    }
    if NP == 0 or M == 0:
        return empty_result

    sig_yy_val = (Y_target.T @ Y_target).item()
    if sig_yy_val < epsilon:
        sig_yy_val = epsilon # Avoid division by zero if Y is zero vector

    selected_terms_indices = []
    err_selected_list = []
    g_selected_list = []
    Q_orthogonal_bases = np.empty((NP, 0))
    A_matrix = np.empty((0,0))

    M0 = 0 # Number of selected terms

    for s_term_iter in range(M): # Iterate at most M times to select M terms

        current_ERRs = np.full(M, -np.inf) # ERR for all M candidate terms in this iteration
        current_gs = np.zeros(M)           # g for all M candidate terms
        current_Qs_storage = np.zeros((NP, M)) # Storage for orthogonalized candidates

        if s_term_iter == 0: # Selecting the first term
            for m_idx in range(M):
                p_m = P_regressors[:, m_idx:m_idx+1]
                p_m_norm_sq = (p_m.T @ p_m).item()
                if p_m_norm_sq >= epsilon:
                    # For the first term, Q_m = P_m
                    current_Qs_storage[:,m_idx] = p_m.flatten() # Store P_m as Q_m
                    current_gs[m_idx] = (Y_target.T @ p_m).item() / p_m_norm_sq
                    current_ERRs[m_idx] = (current_gs[m_idx]**2 * p_m_norm_sq) / sig_yy_val
        else: # Selecting subsequent terms (s_term_iter > 0)
            for m_idx in range(M):
                if m_idx in selected_terms_indices: # Skip already selected terms
                    continue

                p_m = P_regressors[:, m_idx:m_idx+1]
                q_m_orth = p_m.copy() # Start with P_m

                # Orthogonalize P_m against previously selected Q_orthogonal_bases
                for r_q_idx in range(M0): # M0 is number of terms already in Q_orthogonal_bases
                    q_r = Q_orthogonal_bases[:, r_q_idx:r_q_idx+1]
                    q_r_norm_sq = (q_r.T @ q_r).item()
                    alpha_mr = 0.0
                    if q_r_norm_sq >= epsilon:
                        alpha_mr = (p_m.T @ q_r).item() / q_r_norm_sq
                    q_m_orth -= alpha_mr * q_r

                current_Qs_storage[:,m_idx] = q_m_orth.flatten()
                q_m_orth_norm_sq = (q_m_orth.T @ q_m_orth).item()

                if q_m_orth_norm_sq >= epsilon:
                    current_gs[m_idx] = (Y_target.T @ q_m_orth).item() / q_m_orth_norm_sq
                    current_ERRs[m_idx] = (current_gs[m_idx]**2 * q_m_orth_norm_sq) / sig_yy_val

        if np.all(np.isneginf(current_ERRs)): # No more terms can be selected
            break

        newly_selected_idx = np.argmax(current_ERRs)

        # Store results for the newly selected term
        selected_terms_indices.append(newly_selected_idx)
        err_selected_list.append(current_ERRs[newly_selected_idx])
        g_selected_list.append(current_gs[newly_selected_idx])

        Q_selected_term = current_Qs_storage[:, newly_selected_idx:newly_selected_idx+1]
        if Q_orthogonal_bases.shape[1] == 0: # First term
            Q_orthogonal_bases = Q_selected_term
        else:
            Q_orthogonal_bases = np.hstack((Q_orthogonal_bases, Q_selected_term))

        # Update A matrix
        p_original_newly_selected = P_regressors[:, newly_selected_idx:newly_selected_idx+1]
        if M0 == 0: # First term selected
            A_matrix = np.array([[1.0]])
        else:
            A_new_col = np.zeros((M0, 1))
            for r_A_idx in range(M0):
                q_r_for_A = Q_orthogonal_bases[:, r_A_idx:r_A_idx+1] # These are from previous Qs
                q_r_for_A_norm_sq = (q_r_for_A.T @ q_r_for_A).item()
                if q_r_for_A_norm_sq >= epsilon:
                    A_new_col[r_A_idx, 0] = (p_original_newly_selected.T @ q_r_for_A).item() / q_r_for_A_norm_sq

            A_matrix = np.block([
                [A_matrix, A_new_col],
                [np.zeros((1, M0)), np.array([[1.0]])]
            ])
        M0 += 1 # Increment number of selected terms

        # Check stopping criterion
        esr = 1.0 - np.sum(err_selected_list)
        if esr <= rho:
            break

    if M0 == 0: # No terms were selected at all
        return empty_result

    # Final parameter calculation: A * theta = g
    A_final = A_matrix # This is M0 x M0
    g_final = np.array(g_selected_list).reshape(-1, 1) # M0 x 1

    theta_FROLS = np.array([])
    if A_final.size > 0:
        try:
            theta_FROLS = np.linalg.solve(A_final, g_final)
        except np.linalg.LinAlgError: # Should not happen if A is well-formed (upper triangular with 1s)
            theta_FROLS = np.linalg.pinv(A_final) @ g_final # Fallback

    P_sel_data_final = P_regressors[:, selected_terms_indices]
    P_sel_colnames_final = [P_colnames[i] for i in selected_terms_indices] if P_colnames else []

    return {
        'th': theta_FROLS.flatten(),
        'Psel_data': P_sel_data_final,
        'Psel_colnames': P_sel_colnames_final,
        'g': g_final.flatten(),
        'W': Q_orthogonal_bases, # Orthogonal basis matrix
        'A': A_final,
        'ERR_values': np.array(err_selected_list).flatten(),
        'selected_indices': selected_terms_indices
    }

## Testing FROLS with synthetic data

Below we generate synthetic data and then apply FROLS to select the terms.

We expect to recover the list of candidate variables as in the synthetic model.



In [15]:
# 1. Generate simulation data ------------------------------------------------
N = 200
np.random.seed(0) # For reproducibility, optional
u_sim = np.random.uniform(low=-1, high=1, size=N)
e_sim = np.random.normal(loc=0, scale=0.1, size=N)
y_sim = np.zeros(N)

# NARX process:
# y[k] = -0.605*y[k-1] - 0.163*y[k-2]^2 + 0.588*u[k-1] - 0.24*u[k-2] + e[k]
# Indices for y_sim and u_sim are 0-based.
# k_idx corresponds to current time 'k'.
# y_sim[k_idx-1] is y(k-1), y_sim[k_idx-2] is y(k-2)
# u_sim[k_idx-1] is u(k-1), u_sim[k_idx-2] is u(k-2)
for k_idx in range(2, N): # Start from k_idx=2 (3rd element, time k=3 if 1-indexed)
    y_sim[k_idx] = (
        -0.605 * y_sim[k_idx-1]
        - 0.163 * (y_sim[k_idx-2]**2)
        + 0.588 * u_sim[k_idx-1]
        - 0.24 * u_sim[k_idx-2]
        + e_sim[k_idx]
    )

# 2. Model parameters --------------------------------------------------------
rho_threshold = 0.03 # ESR < 3%
nu_order = 2
ny_order = 2
# ne_order = 0 # Not explicitly used in regMatNARX in this context
poly_degree_l = 3

# 3. Create regression and target matrices -----------------------------------
# Python regMatNARX takes (u, y, nu, ny, l) and returns P, P_colnames, Y_target
P_candidate_matrix, P_candidate_colnames, Y_target_vector = regMatNARX(
    u_sim, y_sim, nu_order, ny_order, poly_degree_l
)

# 4. FROLS term selection and parameter estimation ---------------------------
frols_results = frols(
    P_candidate_matrix,
    Y_target_vector,
    rho_threshold,
    P_candidate_colnames
)

# 5. Print important information ---------------------------------------------
print("--- FROLS Results ---")
print("\nSelected terms:")
if frols_results['Psel_colnames']:
    for i, term_name in enumerate(frols_results['Psel_colnames']):
        print(f"  {i+1}. {term_name}")
else:
    print("  No terms selected.")

print("\nEstimated parameters (theta):")
if frols_results['th'].size > 0:
    for i, param_val in enumerate(frols_results['th']):
        term_name = frols_results['Psel_colnames'][i] if frols_results['Psel_colnames'] else f"Term {i+1}"
        print(f"  {term_name}: {param_val:.4f}")
else:
    print("  No parameters estimated.")

print("\nERR values for selected terms (%):")
if frols_results['ERR_values'].size > 0:
    for i, err_val in enumerate(frols_results['ERR_values']):
        term_name = frols_results['Psel_colnames'][i] if frols_results['Psel_colnames'] else f"Term {i+1}"
        print(f"  {term_name}: {err_val * 100:.2f}%")
else:
    print("  No ERR values.")

# You can access other results like:
# frols_results['Psel_data'] # Matrix of selected regressors
# frols_results['W']         # Orthogonal basis matrix
# frols_results['A']         # Transformation matrix A
# frols_results['g']         # Parameters g for orthogonal model
# frols_results['selected_indices'] # 0-based indices of selected terms from P_candidate_matrix

--- FROLS Results ---

Selected terms:
  1. y(k-1)
  2. u(k-1)
  3. y(k-2)y(k-2)
  4. u(k-2)

Estimated parameters (theta):
  y(k-1): -0.6150
  u(k-1): 0.5739
  y(k-2)y(k-2): -0.1872
  u(k-2): -0.2361

ERR values for selected terms (%):
  y(k-1): 69.66%
  u(k-1): 21.30%
  y(k-2)y(k-2): 4.77%
  u(k-2): 2.37%


The results are comparable with Table 3.3 in Billings' book. We used herein ESR (rho) of 3%.